# Playground for parsing and extracting information from PDFs - Llama Index


## Parsing PDF document using LlamaParse

LlamaParse is a highly accurate parser for complex documents like financial reports, research papers, and scanned PDFs. It handles images, tables, and charts wiht ease.

It transforms complex documents into text, markdown, or JSON formats.

[Example](https://github.com/run-llama/llama_cloud_services/blob/main/parse.md)

In [20]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [6]:
from llama_cloud_services import LlamaParse
import os

faq_file = os.path.abspath("./novo_regime.pdf")

parser = LlamaParse(result_type="text", api_key=os.getenv("LLAMA_CLOUD_API_KEY", None))

results = parser.parse(faq_file)

results


Started parsing the file under job_id 526bd40a-cef4-4f9c-9887-9bdbd74133e4


JobResult(pages=[Page(page=1, text='O limite mínimo da base de incidência contributiva dos trabalhadores independentes abrangidos pelo\nregime de contabilidade organizada é 1,5 vezes o valor do IAS.\nnovos\n\n\nPerguntas Frequentes\n\n\nNOVO REGIME DOS TRABALHADORES INDEPENDENTES\n\nINSTITUTO DA SEGURANÇA SOCIAL, I.P\n\n\nSEGURANCA SOCIAL    is\n                    INSTITUTO DA SEGURANCA SOCIAL, L.P:', md='\n# NOVO REGIME DOS TRABALHADORES INDEPENDENTES\n\n# INSTITUTO DA SEGURANÇA SOCIAL, I.P\n\nO limite mínimo da base de incidência contributiva dos trabalhadores independentes abrangidos pelo regime de contabilidade organizada é 1,5 vezes o valor do IAS.\n\n# Perguntas Frequentes\n\nSEGURANCA SOCIAL    is\n\nINSTITUTO DA SEGURANCA SOCIAL, L.P:\n\n', images=[ImageItem(name='img_p0_1.png', height=2339.0, width=1654.0, x=-3.8498, y=0.7497999999999365, original_width=1654, original_height=2339, type=None), ImageItem(name='img_p0_2.png', height=507.0, width=1468.0, x=44.16, y=266.0399999999

In [15]:
text_documents = results.get_text_documents(split_by_page=True)

text_documents

[Document(id_='50363e79-2bc1-4683-aa9b-dad71fafd6c4', embedding=None, metadata={'page_number': 1, 'file_name': '/Users/patriciaoliveira/repos/ai-assistant-self-employers-pt/notebooks/novo_regime.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='O limite mínimo da base de incidência contributiva dos trabalhadores independentes abrangidos pelo\nregime de contabilidade organizada é 1,5 vezes o valor do IAS.\nnovos\n\n\nPerguntas Frequentes\n\n\nNOVO REGIME DOS TRABALHADORES INDEPENDENTES\n\nINSTITUTO DA SEGURANÇA SOCIAL, I.P\n\n\nSEGURANCA SOCIAL    is\n                    INSTITUTO DA SEGURANCA SOCIAL, L.P:', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='1f6d61d4-b06c-4076-99d4-26b4a8014138', embedding=None, metadat

## Indexing documents

Using an in-memory vector database (`VectorStoreIndex`) and a fully-managed vector database from LlmaCloud (`LlamaCloudIndex`).

Both classes - `VectorStoreIndex` and `LlamaCloudIndex` - inherit behavior from `BaseIndex` class, such as the ability to create a retriever, a query engine supported by a LLM, and a chat engine.

- `as_retriever()`: it creates a retriever from the index.

- `as_query_engine()`: it converts the index to a query engine.

- `as_chat_engine()`: it converts the index to a chat engine.

### Using `VectorStoreIndex`


`VectorStoreIndex` uses a in-memory `SimpleVectorStore` that is initialized as part of the default storage context.

[Documentation](https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/)


In [24]:
from llama_index.core import VectorStoreIndex

# Create an index from the documents. The VectorStoreIndex converts the parsed documents into a vectorized format that can be queried.
# This index will allow us to perform semantic searches over the content of the documents.
local_index = VectorStoreIndex.from_documents(text_documents)

local_index

In [25]:
# This retriever allows us to perform searches on the index using natural language queries.
retriever = local_index.as_retriever()

# It returns the nodes that match the query.
nodes = retriever.retrieve("Como é determinado o rendimento?")

nodes

[NodeWithScore(node=TextNode(id_='7ddff3cf-245b-4a83-af14-063051b6003c', embedding=None, metadata={'page_number': 14, 'file_name': '/Users/patriciaoliveira/repos/ai-assistant-self-employers-pt/notebooks/novo_regime.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5c5a0078-b1a3-4b4a-9d1e-2c75678682f9', node_type='4', metadata={'page_number': 14, 'file_name': '/Users/patriciaoliveira/repos/ai-assistant-self-employers-pt/notebooks/novo_regime.pdf'}, hash='e26971de93f555551e8ee6f4a1bdbb4be519da697a96340abc99fc2d278dbee3')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Perguntas Frequentes | Novo Regime dos Trabalhadores Independentes\n\ndeclaração de rendimentos correspondente ao ano de 2023, ano em que o TI mudou de regime\npara o de contabilidade organizada.\n\n\nD – DETERMINAÇÃO DO RENDIMENTO RELEVANTE\n\n24. Como é determinado o rendimento relevante do trabalhador indepen

In [26]:
# It creates a query search engine from the index.
# This engine will allow us to query the index using natural language questions and returns the most relevant results.
query_engine = local_index.as_query_engine()

response = query_engine.query("Como é determinado o rendimento?")

response.response

'O rendimento é determinado com base nos rendimentos obtidos nos três meses imediatamente anteriores ao mês da declaração trimestral, de acordo com percentagens específicas para diferentes tipos de rendimentos. Para trabalhadores independentes abrangidos pelo regime de contabilidade organizada, o rendimento relevante corresponde ao lucro tributável apurado no ano civil imediatamente anterior, declarado no Anexo SS da Declaração Modelo 3 do IRS.'

### Accessing prompts running in the background

In [27]:

prompts = query_engine.get_prompts()

for k, q in prompts.items():
    print(f"Prompt for {k}:")
    print(q)
    print("\n")

Prompt for response_synthesizer:text_qa_template:
metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>} template_vars=['context_str', 'query_str'] kwargs={} output_parser=None template_var_mappings={} function_mappings={} default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: ') conditionals=[(<function is_chat_model at 0x10f4f1ee0>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additi

### Using `LlamaCloudIndex`

LlmaCloud Index offers a full-managed option to store vectorized data in a vector database. It is also possible to connect our database instances in LlamaCloud.

[Documentation](https://docs.llamaindex.ai/en/stable/module_guides/indexing/llama_cloud_index/)

[Example](https://github.com/run-llama/llama_cloud_services/blob/main/index.md)

In [16]:
from llama_cloud_services import LlamaCloudIndex

# It creates a new LlamaCloudIndex instance
cloud_index = LlamaCloudIndex.from_documents(
    documents=text_documents,
    name="faqs_index__example",
    verbose=True,
)

cloud_index

Created project 2a57e3bb-7adb-4789-9321-ea85439c63a8 with name Default
Created pipeline e1388492-979a-430d-8993-9ed7b655cb1d with name faqs_index__example
Loading documents
Document ingestion finished for 3d7b220f-6411-48c2-9f6f-3c2d1959fc0a
Document ingestion finished for 9dee5aa2-1671-45d1-b941-2adfa168c38e
Document ingestion finished for 1f6d61d4-b06c-4076-99d4-26b4a8014138
Document ingestion finished for 74975017-c60e-4a9d-a688-a767781dbae0
Document ingestion finished for ed8e0abb-f76c-4f3d-a58a-b3663acc3a60
Document ingestion finished for adfc6c61-ac1b-4377-b814-444e4d124f0a
Document ingestion finished for 3233089b-e9ac-4d1f-8ce7-3ee75ad33bfb
Document ingestion finished for 2bce62f5-d893-4b40-8e6d-832e2bc000c3
Document ingestion finished for b9b9f0d8-7923-4ee9-bb3a-99b6f659fd04
Document ingestion finished for 8c53a489-f243-4b87-8049-f0e00d548544
Document ingestion finished for 1e350dd9-e344-47d3-b9fc-3b71b50cbe05
Document ingestion finished for 17b6ed01-a8c9-46ea-8371-b224519cb438

In [17]:
# This retriever allows us to perform searches on the index using natural language queries.
retriever = cloud_index.as_retriever()

# It returns the nodes that match the query.
nodes = retriever.retrieve("Como é determinado o rendimento?")

nodes

[NodeWithScore(node=TextNode(id_='3c10d75a-2f03-41ab-880c-66670a183efa', embedding=None, metadata={'page_number': 14, 'file_name': '/Users/patriciaoliveira/repos/ai-assistant-self-employers-pt/notebooks/novo_regime.pdf', 'pipeline_id': 'e1388492-979a-430d-8993-9ed7b655cb1d', 'document_23_page_label': 1, 'document_id': '5c5a0078-b1a3-4b4a-9d1e-2c75678682f9'}, excluded_embed_metadata_keys=['document_id', 'file_id', 'pipeline_file_id'], excluded_llm_metadata_keys=['document_id', 'file_id', 'pipeline_file_id'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5c5a0078-b1a3-4b4a-9d1e-2c75678682f9', node_type='4', metadata={'page_number': 14, 'file_name': '/Users/patriciaoliveira/repos/ai-assistant-self-employers-pt/notebooks/novo_regime.pdf', 'pipeline_id': 'e1388492-979a-430d-8993-9ed7b655cb1d'}, hash='d5cb609e412f33892044c2007711ba6c3f3a519e864971160022dcb16cc21ef9')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Perguntas Frequentes | Novo Reg

In [22]:
# It creates a query search engine from the index.
query_engine = cloud_index.as_query_engine()

# This engine will allow us to query the index using natural language questions and returns the most relevant results.
response = query_engine.query("Como é determinado o rendimento?")

response.response

'O rendimento relevante do trabalhador independente é determinado com base nos rendimentos obtidos nos três meses imediatamente anteriores ao mês da Declaração Trimestral, de acordo com percentagens específicas para diferentes tipos de rendimentos, como prestação de serviços, produção e venda de bens, atividades hoteleiras, restauração e bebidas. Para os trabalhadores independentes abrangidos pelo regime de contabilidade organizada, o rendimento relevante corresponde ao lucro tributável apurado no ano civil imediatamente anterior.'

## Extracting information from PDF document using Structured Output

Llama Index framework provides two ways to extract information from PDFs:
1. Llama Cloud - [LlamaExtract](https://docs.cloud.llamaindex.ai/llamaextract/getting_started/python)
2. Structured LLMs - [Structured Data Extraction](https://docs.llamaindex.ai/en/stable/understanding/extraction/structured_llms/)

### Common Dependencies

Pydantic class that represents FAQs structured output.

In [ ]:
from pydantic import BaseModel, Field

class FAQs(BaseModel):
    question: str = Field(description="The question to be answered.")
    answer: str = Field(description="The answer to the question.")

### LlamaExtract

This tool transforms complex documents into well-types structured data with:
- Customizable extraction agents and schemas
- Batch processing capabilities for scale
- Iterative schema development

**LlmaExtract** provides a simple API for extracting structured data from unstructured documents like PDFs, text files, and images.

In [ ]:
from llama_cloud_services import LlamaExtract
import os

In [ ]:
prompt = """
You are an AI that extracts structured data from text. The FAQ contains a list of questions and answers. There are a total of 55 questions, so make sure you extract all question and answers.

Extraction Rules:
1. Every question starts with an incremental number followed by a dot and ends with a question mark.

2. The answer to that question starts with R, (letter "R" followed by a comma) and continues until the next question or the end of the document.

3. Return a JSON array where each element has:
- question: the full question text.
- answer: the full answer text.

Extract ONLY the questions and answers following these rules. Ignore any other text.
"""

extractor = LlamaExtract()
agents = [agent.name for agent in extractor.list_agents()]

if not "FAQs Seguranca Social" in agents:
    agent = extractor.create_agent(
        name="FAQs Seguranca Social",
        data_schema=FAQs,
        config={
            "extraction_target": "PER_PAGE",
            "extraction_mode": "MULTIMODAL",
            "multimodal_fast_mode": false,
            "system_prompt": prompt,
            "use_reasoning": false,
            "cite_sources": false,
            "confidence_scores": false,
            "chunk_mode": "PAGE",
            "high_resolution_mode": false,
            "invalidate_cache": false,
            "page_range": "3-27"
        }  
    )

agent = extractor.get_agent("FAQs Seguranca Social")
pdf_path = os.path.abspath("../pdfs/novo_regime.pdf")
result = agent.extract(pdf_path)
print(result.data)

Extracting files: 100%|██████████| 1/1 [00:21<00:00, 21.40s/it]

[{'question': '1. Quem está abrangido pelo novo regime dos trabalhadores independentes?', 'answer': 'R, Estão abrangidos pelo novo regime dos trabalhadores independentes todos os trabalhadores que exerçam atividade por conta própria, incluindo empresários em nome individual, titulares de estabelecimentos individuais de responsabilidade limitada, profissionais liberais, entre outros, que não estejam expressamente excluídos por lei.'}, {'question': '1.  Quando produz efeito o enquadramento no regime?', 'answer': 'R: O primeiro enquadramento no regime dos trabalhadores independentes só produz efeitos no 1.° dia do 12.º mês posterior ao do início de atividade.\n\nExemplo:\n\nSe o trabalhador independente iniciou a sua atividade a 1 de março de 2024, o seu enquadramento produz efeitos a 1 de março de 2025.\n\nSe o trabalhador independente iniciar a sua atividade independente na Autoridade Tributária e Aduaneira a 10 de janeiro de 2024, o seu enquadramento produz efeitos a 1 de janeiro de 20

In [ ]:
for item in result.data:
    print(f"Question: {item['question']}")
    print(f"Answer: {item['answer']}")
    print("-" * 40)

Question: 1. Quem está abrangido pelo novo regime dos trabalhadores independentes?
Answer: R, Estão abrangidos pelo novo regime dos trabalhadores independentes todos os trabalhadores que exerçam atividade por conta própria, incluindo empresários em nome individual, titulares de estabelecimentos individuais de responsabilidade limitada, profissionais liberais, entre outros, que não estejam expressamente excluídos por lei.
----------------------------------------
Question: 1.  Quando produz efeito o enquadramento no regime?
Answer: R: O primeiro enquadramento no regime dos trabalhadores independentes só produz efeitos no 1.° dia do 12.º mês posterior ao do início de atividade.

Exemplo:

Se o trabalhador independente iniciou a sua atividade a 1 de março de 2024, o seu enquadramento produz efeitos a 1 de março de 2025.

Se o trabalhador independente iniciar a sua atividade independente na Autoridade Tributária e Aduaneira a 10 de janeiro de 2024, o seu enquadramento produz efeitos a 1 de 

### Structured Data Extraction

The highest-level way to extract structured data in LlmaIndex is to instantiate a Structured LLM. It refers to the practice of guiding LLMs to produce output in a defined format, like JSON or SML, rather than free-form test.

This structured output is more easily parsed and utilized by other systems, making LLM-driven pipelines more deterministic and efficient.

In [ ]:
from llama_index.readers.file.docs.base import PDFReader
from pathlib import Path

pdf_reader = PDFReader()
documents = pdf_reader.load_data(file=pdf_path)

documents[0].text.strip()

'O limite mínimo da base de incidência contributiva dos trabalhadores independentes abrangidos pelo \nregime de contabilidade organizada é 1,5 vezes o valor do IAS.   \nnovos   \nPerguntas Frequentes \n  \n \nNOVO REGIME DOS TRABALHADORES INDEPENDENTES \n \n \n \nINSTITUTO DA SEGURANÇA SOCIAL, I.P'

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate

llm = OpenAI(model="gpt-4o")
prompt_template = PromptTemplate("You are an AI that extracts structured data from text. The FAQ contains a list of questions and answers: '{document}'. There are a total of 55 questions, so make sure you extract all question and answers. Extraction Rules: Every question starts with an incremental number followed by a dot and ends with a question mark. The answer to that question starts with R, (letter "R" followed by a comma) and continues until the next question or the end of the document.")

response = llm.structured_predict(
    FAQs,
    prompt_template,
    document=documents[0].text.strip()
)